In [3]:
# Imports
# pip install kagglehub[pandas-datasets]
#import kagglehub
#from kagglehub import KaggleDatasetAdapter
import os
import random
from glob import glob
import cv2 
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np


import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

import argparse
#import torch
#import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

import sys
sys.path.append(os.path.abspath("../../../TCC_25"))  # Adiciona o diretório atual ao caminho
print(sys.path)
#sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..', '..', 'TCC_25', 'OPFlmb')))
from OPFlmb.oversampling.o2pf import O2PF
from OPFlmb.undersampling.opf_us import OpfUS
from collections import Counter


['C:\\Users\\giueg\\.pyenv\\pyenv-win\\versions\\3.11.0\\python311.zip', 'C:\\Users\\giueg\\.pyenv\\pyenv-win\\versions\\3.11.0\\DLLs', 'C:\\Users\\giueg\\.pyenv\\pyenv-win\\versions\\3.11.0\\Lib', 'C:\\Users\\giueg\\.pyenv\\pyenv-win\\versions\\3.11.0', 'c:\\Users\\giueg\\.virtualenvs\\TCC_25-OGy0qkdu', '', 'c:\\Users\\giueg\\.virtualenvs\\TCC_25-OGy0qkdu\\Lib\\site-packages', 'c:\\Users\\giueg\\.virtualenvs\\TCC_25-OGy0qkdu\\Lib\\site-packages\\win32', 'c:\\Users\\giueg\\.virtualenvs\\TCC_25-OGy0qkdu\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\giueg\\.virtualenvs\\TCC_25-OGy0qkdu\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\giueg\\OneDrive\\Documentos\\Workspace\\trab_Graduação\\TCC_25\\OPFlmb', 'c:\\Users\\giueg\\OneDrive\\Documentos\\Workspace\\trab_Graduação\\TCC_25']


## Data over tests

In [ ]:
# Tentativa oversampling O2PF
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print(np.array(X).shape)
print(np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print(np.array(x_train).shape)
#print(np.array(x_val).shape)
print(np.array(x_test).shape)
# (3045, 128, 128, 3)
# (0, 128, 128, 3)
# (762, 128, 128, 3)

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Verificar distribuição das classes
print("Distribuição original das classes:", Counter(y_train))

# Achatamento das imagens para O2PF (necessário para formato 2D)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Forma: (3045, 128*128*3)
print("Forma achatada:", x_train_flat.shape)

# Aplicar SMOTE

# Aplicar O2PF
k_max = 5  # Escolha um valor para k_max (exemplo: 5, conforme sugerido no repositório)
o2pf = O2PF(k_max=k_max)
x_res, y_res = o2pf.fit_resample(x_train_flat, y_train)

# Reformatar os dados para o formato de imagem
x_res = x_res.reshape(-1, 128, 128, 3)
print("Forma após reamostragem:", x_res.shape)
print("Distribuição após reamostragem:", Counter(y_res))

# Apply the transform to the data
dataset_train_res = CustomImageDataset(x_res, y_res, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train_res[0]
print(img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print(dataset_train_res.__len__()) # 3045 images
#print(dataset_val.__len__()) # 0 images
print(dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train_res, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


(3807, 128, 128, 3)
(3807,)
(3045, 128, 128, 3)
(762, 128, 128, 3)
Distribuição original das classes: Counter({np.int64(0): 2435, np.int64(2): 402, np.int64(1): 208})
Forma achatada: (3045, 49152)


MemoryError: Unable to allocate 18.0 GiB for an array with shape (49152, 49152) and data type float64

In [ ]:
# Tentativa undersampling O2PF
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print("x = ", np.array(X).shape)
print("Y = ", np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print("\nForma dos dados de treino:", np.array(x_train).shape) # (3045, 128, 128, 3)
#print(np.array(x_val).shape)
print("Forma dos dados de teste:", np.array(x_test).shape) # (762, 128, 128, 3)

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Ajustar rótulos para o OpfUS (>= 1)
y_train_adj = y_train + 1

# Verificar distribuição das classes
print("\nDistribuição original das classes train:", Counter(y_train))
print("Distribuição original das classes train_adj:", Counter(y_train_adj))

# Achatamento das imagens para O2PF (necessário para formato 2D)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Forma: (3045, 128*128*3)
print("\nForma achatada:", x_train_flat.shape)

# Separar as classes para processar 

# Aplicar OpfUS
opf_us = OpfUS()
x_res, y_res = opf_us.fit_resample(x_train_flat, y_train_adj)

print("\nForma antes reamostragem:", x_res.shape)
print("Distribuição antes reamostragem:", Counter(y_res))

# Corrigir os rótulos de volta para começar em 0 (como o resto da pipeline espera)
y_res_new = y_res - 1

# Reformatar os dados para o formato de imagem
x_res_new = x_res.reshape(-1, 128, 128, 3)
print("\nForma após reamostragem:", x_res.shape)
print("Distribuição após reamostragem:", Counter(y_res_new))

# Apply the transform to the data
dataset_train_res = CustomImageDataset(x_res_new, y_res_new, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train_res[0]
print("\nForma do img e label:", img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print("Quantidade dados de treino:", dataset_train_res.__len__()) # 3045 images
#print(dataset_val.__len__()) # 0 images
print("Quantidade dados de teste:", dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train_res, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


x =  (3807, 128, 128, 3)
Y =  (3807,)

Forma dos dados de treino: (3045, 128, 128, 3)
Forma dos dados de teste: (762, 128, 128, 3)

Distribuição original das classes train: Counter({np.int64(0): 2435, np.int64(2): 402, np.int64(1): 208})
Distribuição original das classes train_adj: Counter({np.int64(1): 2435, np.int64(3): 402, np.int64(2): 208})

Forma achatada: (3045, 49152)

Forma antes reamostragem: (610, 49152)
Distribuição antes reamostragem: Counter({np.int64(3): 402, np.int64(2): 208})

Forma após reamostragem: (610, 49152)
Distribuição após reamostragem: Counter({np.int64(2): 402, np.int64(1): 208})

Forma do img e label: torch.Size([1, 128, 128]) 2
Quantidade dados de treino: 610
Quantidade dados de teste: 762


In [9]:
y_test_adj = y_test + 1

In [12]:
print("TEST")
print("Rótulos de teste ajustados:", x_test.shape)
print("Distribuição rótulos de teste ajustados (adj):", Counter(y_test_adj))
print("Distribuição rótulos de teste ajustados :", Counter(y_test))

print("TRAIN")
print("Rótulos de teste ajustados:", x_res_new.shape)
print("Distribuição rótulos de teste ajustados (new):", Counter(y_res_new))
print("Distribuição rótulos de teste ajustados:", Counter(y_res))


TEST
Rótulos de teste ajustados: (762, 128, 128, 3)
Distribuição rótulos de teste ajustados (adj): Counter({np.int64(1): 609, np.int64(3): 101, np.int64(2): 52})
Distribuição rótulos de teste ajustados : Counter({np.int64(0): 609, np.int64(2): 101, np.int64(1): 52})
TRAIN
Rótulos de teste ajustados: (610, 128, 128, 3)
Distribuição rótulos de teste ajustados (new): Counter({np.int64(2): 402, np.int64(1): 208})
Distribuição rótulos de teste ajustados: Counter({np.int64(3): 402, np.int64(2): 208})


In [ ]:
#salvar novo dataset
np.savez("../../data/x_images_arrays_under.npz", arr_0=x_res)
np.savez("../../data/y_labels_arrays_under.npz", arr_0=y_res)

In [ ]:
# Tentativa undersampling O2PF
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz_r = np.load("../../data/x_images_arrays_resampled.npz")
X = x_npz_r["arr_0"]
y_npz_r = np.load("../../data/y_labels_arrays_resampled.npz")
Y = y_npz_r["arr_0"]

print("x = ", np.array(X).shape)
print("Y = ", np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)
#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print("\nForma dos dados de treino:", np.array(x_train).shape) # (3045, 128, 128, 3)
print("Counter train:", Counter(y_train))
#print(np.array(x_val).shape)
print("Forma dos dados de teste:", np.array(x_test).shape) # (762, 128, 128, 3)
print("Counter test:", Counter(y_test))

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Apply the transform to the data
dataset_train = CustomImageDataset(x_train, y_train, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train[0]
print("\nForma do img e label:", img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print("Quantidade dados de treino:", dataset_train.__len__()) # 3045 images
#print(dataset_val.__len__()) # 0 images
print("Quantidade dados de teste:", dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

x =  (610, 128, 128, 3)
Y =  (610,)

Forma dos dados de treino: (488, 128, 128, 3)
Counter train: Counter({np.int64(2): 322, np.int64(1): 166})
Forma dos dados de teste: (122, 128, 128, 3)
Counter test: Counter({np.int64(2): 80, np.int64(1): 42})

Forma do img e label: torch.Size([1, 128, 128]) 2
Quantidade dados de treino: 488
Quantidade dados de teste: 122


## Under

In [ ]:
# Tentativa undersampling O2PF
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print("x = ", np.array(X).shape)
print("Y = ", np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print("\nForma dos dados de treino:", np.array(x_train).shape) # (3045, 128, 128, 3)
print("Distribuição train antes:", Counter(y_train)) #np.int64(0): 2435, np.int64(2): 402, np.int64(1): 208
#print(np.array(x_val).shape)
print("Forma dos dados de teste:", np.array(x_test).shape) # (762, 128, 128, 3)
print("Distribuição test antes:", Counter(y_test)) #np.int64(0): 609, np.int64(2): 101, np.int64(1): 52

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Índices de cada classe
idx_class_0 = np.where(y_train == 0)[0]
idx_class_1 = np.where(y_train == 1)[0]
idx_class_2 = np.where(y_train == 2)[0]

# Undersample da classe 1 para ficar com, por exemplo, o mesmo número da classe 3
n_samples = len(idx_class_2)  # ou defina outro número
np.random.seed(42)
idx_class_0_sampled = np.random.choice(idx_class_0, size=n_samples, replace=False)

# Combinar os índices finais
final_indices = np.concatenate([idx_class_0_sampled, idx_class_1, idx_class_2])
np.random.shuffle(final_indices)

# Aplicar seleção nos dados
x_train_resampled = x_train[final_indices]
y_train_resampled = y_train[final_indices]

print("\nForma da trian após reamostragem:", x_train_resampled.shape) #(1012, 128, 128, 3)
print("Distribuição train após reamostragem:", Counter(y_train_resampled)) #np.int64(2): 402, np.int64(0): 402, np.int64(1): 208

x =  (3807, 128, 128, 3)
Y =  (3807,)

Forma dos dados de treino: (3045, 128, 128, 3)
Distribuição train antes: Counter({np.int64(0): 2435, np.int64(2): 402, np.int64(1): 208})
Forma dos dados de teste: (762, 128, 128, 3)
Distribuição test antes: Counter({np.int64(0): 609, np.int64(2): 101, np.int64(1): 52})

Forma da trian após reamostragem: (1012, 128, 128, 3)
Distribuição train após reamostragem: Counter({np.int64(2): 402, np.int64(0): 402, np.int64(1): 208})


In [ ]:
# Apply the transform to the data
dataset_train_res = CustomImageDataset(x_res_new, y_train_resampled, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train_res[0]
print("\nForma do img e label:", img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print("Quantidade dados de treino:", dataset_train_res.__len__()) # 610 images
#print(dataset_val.__len__()) # 0 images
print("Quantidade dados de teste:", dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train_res, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


Forma do img e label: torch.Size([1, 128, 128]) 1
Quantidade dados de treino: 610
Quantidade dados de teste: 762


In [19]:
#salvar novo dataset
np.savez("../../data/x_images_arrays_under_train.npz", arr_0=x_train_resampled)
np.savez("../../data/y_labels_arrays_under_train.npz", arr_0=y_train_resampled)

## Over

In [ ]:
# Tentativa oversampling O2PF
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print(np.array(X).shape)
print(np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print(np.array(x_train).shape)
#print(np.array(x_val).shape)
print(np.array(x_test).shape)
# (3045, 128, 128, 3)
# (0, 128, 128, 3)
# (762, 128, 128, 3)

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Verificar distribuição das classes
print("Distribuição original das classes:", Counter(y_train))

# Achatamento das imagens para O2PF (necessário para formato 2D)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Forma: (3045, 128*128*3)
print("Forma achatada:", x_train_flat.shape)

# Aplicar SMOTE

# Aplicar O2PF
k_max = 5  # Escolha um valor para k_max (exemplo: 5, conforme sugerido no repositório)
o2pf = O2PF(k_max=k_max)
x_res, y_res = o2pf.fit_resample(x_train_flat, y_train)

# Reformatar os dados para o formato de imagem
x_res = x_res.reshape(-1, 128, 128, 3)
print("Forma após reamostragem:", x_res.shape)
print("Distribuição após reamostragem:", Counter(y_res))

# Apply the transform to the data
dataset_train_res = CustomImageDataset(x_res, y_res, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train_res[0]
print(img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print(dataset_train_res.__len__()) # 3045 images
#print(dataset_val.__len__()) # 0 images
print(dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train_res, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [6]:
# Tentativa oversampling O2PF depois do undersampling
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print("x = ", np.array(X).shape)
print("Y = ", np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train_old, x_test, y_train_old, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

x_train_aux = np.load("../../data/x_images_arrays_under_train.npz")
x_train = x_train_aux["arr_0"]
y_train_aux = np.load("../../data/y_labels_arrays_under_train.npz")
y_train = y_train_aux["arr_0"]

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print("\nForma dos dados de treino:", np.array(x_train).shape) # (1012, 128, 128, 3)
print("Distribuição train antes:", Counter(y_train)) #{np.int64(2): 402, np.int64(0): 402, np.int64(1): 208}
#print(np.array(x_val).shape)
print("Forma dos dados de teste:", np.array(x_test).shape) # (762, 128, 128, 3)
print("Distribuição test antes:", Counter(y_test)) #np.int64(0): 609, np.int64(2): 101, np.int64(1): 52

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# # Achatamento das imagens para O2PF (necessário para formato 2D)
# x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Forma: (1012, 128*128*3)
# print("Forma achatada:", x_train_flat.shape)

# # Aplicar O2PF
# k_max = 5  # Escolha um valor para k_max (exemplo: 5, conforme sugerido no repositório)
# o2pf = O2PF(k_max=k_max)
# x_res, y_train_res = o2pf.fit_resample(x_train_flat, y_train)

# Achatamento das imagens para O2PF (necessário para formato 2D)
x_train_flat = x_train.reshape(x_train.shape[0], -1)  # Forma: (1012, 128*128*3)
print("Forma achatada:", x_train_flat.shape)

# Filtrar apenas a classe 1 (208 amostras)
mask_class1 = y_train == 1
x_train_class1 = x_train_flat[mask_class1]  # Forma: (208, 128*128*3)
y_train_class1 = y_train[mask_class1]  # Forma: (208,)

# Aplicar O2PF apenas à classe 1
k_max = 5  # Ajuste conforme necessário
o2pf = O2PF(k_max=k_max)
x_res_class1, y_res_class1 = o2pf.fit_resample(x_train_class1, y_train_class1)

# Verificar a nova distribuição da classe 1
print("Distribuição da classe 1 após O2PF:", Counter(y_res_class1))

# Ajustar manualmente para 402 amostras, se necessário
desired_samples = 402
current_samples = len(y_res_class1)
if current_samples > desired_samples:
    # Reduzir para 402 amostras (seleção aleatória)
    indices = np.random.choice(current_samples, desired_samples, replace=False)
    x_res_class1 = x_res_class1[indices]
    y_res_class1 = y_res_class1[indices]
elif current_samples < desired_samples:
    # Aumentar com duplicação aleatória, se necessário
    additional_samples = desired_samples - current_samples
    indices = np.random.choice(current_samples, additional_samples, replace=True)
    x_res_class1 = np.concatenate([x_res_class1, x_res_class1[indices]], axis=0)
    y_res_class1 = np.concatenate([y_res_class1, y_res_class1[indices]], axis=0)

print("Distribuição da classe 1 após ajuste:", Counter(y_res_class1))

# Combinar os dados reamostrados da classe 1 com as classes 0 e 2
x_train_other = x_train_flat[~mask_class1]  # Classes 0 e 2
y_train_other = y_train[~mask_class1]
x_res = np.concatenate([x_train_other, x_res_class1], axis=0)
y_train_res = np.concatenate([y_train_other, y_res_class1], axis=0)

# Reformatar os dados para o formato de imagem
x_train_res = x_res.reshape(-1, 128, 128, 3)

print("\nForma dos dados de treino:", np.array(x_train_res).shape) # 
print("Distribuição train antes:", Counter(y_train_res)) #
#print(np.array(x_val).shape)
print("Forma dos dados de teste:", np.array(x_test).shape) # (762, 128, 128, 3)
print("Distribuição test antes:", Counter(y_test)) #np.int64(0): 609, np.int64(2): 101, np.int64(1): 52

# Apply the transform to the data
dataset_train = CustomImageDataset(x_train_res, y_train_res, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train[0]
print("\nForma do img e label:", img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print("Quantidade dados de treino:", dataset_train.__len__()) # 610 images
#print(dataset_val.__len__()) # 0 images
print("Quantidade dados de teste:", dataset_test.__len__()) # 762 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


x =  (3807, 128, 128, 3)
Y =  (3807,)

Forma dos dados de treino: (1012, 128, 128, 3)
Distribuição train antes: Counter({np.int64(2): 402, np.int64(0): 402, np.int64(1): 208})
Forma dos dados de teste: (762, 128, 128, 3)
Distribuição test antes: Counter({np.int64(0): 609, np.int64(2): 101, np.int64(1): 52})
Forma achatada: (1012, 49152)
Distribuição da classe 1 após O2PF: Counter({np.int64(1): 208})
Distribuição da classe 1 após ajuste: Counter({np.int64(1): 402})

Forma dos dados de treino: (1206, 128, 128, 3)
Distribuição train antes: Counter({np.int64(2): 402, np.int64(0): 402, np.int64(1): 402})
Forma dos dados de teste: (762, 128, 128, 3)
Distribuição test antes: Counter({np.int64(0): 609, np.int64(2): 101, np.int64(1): 52})

Forma do img e label: torch.Size([1, 128, 128]) 2
Quantidade dados de treino: 1206
Quantidade dados de teste: 762


In [7]:
#salvar novo dataset
np.savez("../../data/x_images_arrays_under_train_over.npz", arr_0=x_train_res)
np.savez("../../data/y_labels_arrays_under_train_over.npz", arr_0=y_train_res)